In [25]:
from pymongo import MongoClient
import pandas as pd

client = MongoClient()
db = client.soaeng
dbcol_subtitle = db.subtitle
dbcol_ani_voca = db.ani_voca
dbcol_info = db.ani_info
dbcol_words = db.words

## ✨ 나루토
### 나루토 자막만 가져오기

In [ ]:
query = {
        "filename": {"$regex": "Naruto"}
}
df_words = pd.DataFrame(dbcol_subtitle.find(query))
df_words = df_words[["frequency"]]
df_words

In [ ]:
# 나루토
naruto_series = 3907
df_naruto = pd.DataFrame(dbcol_info.find({"series_id": naruto_series}))
df_naruto[["id", "name"]]

In [ ]:
naruto_id = df_naruto.id.values.tolist()
naruto_id

### 단어: 빈도수 합계

In [ ]:
voca = dict()
def init_words(data):
    frequency = data["frequency"]
    for word, cnt in frequency.items():
        voca[word] = 0
    return voca

def count_words(data, voca=voca):
    frequency = data["frequency"]
    for word, cnt in frequency.items():
        voca[word] += cnt
    return voca

df_words.apply(init_words, axis=1)
df_words.apply(count_words, axis=1)

### 빈도수 순으로 1000개

In [ ]:
print(len(voca))
voca_list = sorted(voca.items(), key=lambda x: x[1], reverse=True)[:1000]
print(len(voca_list))
voca_list

### df화, 열 지정

In [ ]:
df_voca = pd.DataFrame(voca_list)
df_voca.columns = ["japanese", "frequency"]
df_voca

### 단어 저장 document 생성 후 db 저장

In [ ]:
word_list = []
def save_word(data):
    words = {"japanese": data["japanese"], "frequency":data["frequency"]}
    word_list.append(words)
df_voca.apply(save_word, axis=1)
word_list
dbcol_ani_voca.update_one({"series_id": naruto_series}, {"$set": {"ani_id": naruto_id, "words": word_list}}, upsert = True)

### 삽입된 단어 확인

In [ ]:
naruto_voca = dbcol_ani_voca.find_one({"series_id": naruto_series})["words"]
naruto_voca

### 번역, 발음 등 정보 추가

In [ ]:
dbcol_words = db.words
result = []
for w in naruto_voca:
    j = w["japanese"]
    f = w["frequency"]
    word = dbcol_words.find_one({"japanese": j})
    if word["korean"] != word["papago"]:
        word["korean"] = word["korean"] + ", " + word["papago"]
    word = {
        "japanese": word["japanese"],
        "pronunciation": word["pronunciation"],
        "korean": word["korean"],
        "frequency": f
    }
    result.append(word)
result

### 데이터 저장

In [ ]:
dbcol_ani_voca.update_one({"series_id": naruto_series}, {"$set": {"words": result}})

<br/><hr/>

## ✨ 포켓몬스터

In [26]:
query = {
    "$or": [
        {"filename": {"$regex": "Pocket_Monsters"}},
        {"filename": {"$regex": "Pokemon"}},
    ]
}
df_words = pd.DataFrame(dbcol_subtitle.find(query))
df_words = df_words[["frequency"]]
df_words

# 포켓몬
pocket_series = 3485
df_ani = pd.DataFrame(dbcol_info.find({"series_id": pocket_series}))

pocket_id = df_ani.id.values.tolist()

In [28]:
voca = dict()
def init_words(data):
    frequency = data["frequency"]
    for word, cnt in frequency.items():
        voca[word] = 0
    return voca

def count_words(data, voca=voca):
    frequency = data["frequency"]
    for word, cnt in frequency.items():
        voca[word] += cnt
    return voca

df_words.apply(init_words, axis=1)
df_words.apply(count_words, axis=1)

0      {'為る': 19535, 'ポケモン': 7850, '映画': 650, '言う': 7...
1      {'為る': 19535, 'ポケモン': 7850, '映画': 650, '言う': 7...
2      {'為る': 19535, 'ポケモン': 7850, '映画': 650, '言う': 7...
3      {'為る': 19535, 'ポケモン': 7850, '映画': 650, '言う': 7...
4      {'為る': 19535, 'ポケモン': 7850, '映画': 650, '言う': 7...
                             ...                        
579    {'為る': 19535, 'ポケモン': 7850, '映画': 650, '言う': 7...
580    {'為る': 19535, 'ポケモン': 7850, '映画': 650, '言う': 7...
581    {'為る': 19535, 'ポケモン': 7850, '映画': 650, '言う': 7...
582    {'為る': 19535, 'ポケモン': 7850, '映画': 650, '言う': 7...
583    {'為る': 19535, 'ポケモン': 7850, '映画': 650, '言う': 7...
Length: 584, dtype: object

In [29]:
print(len(voca))
voca_list = sorted(voca.items(), key=lambda x: x[1], reverse=True)[:1000]
print(len(voca_list))
voca_list

16459
1000


[('為る', 19535),
 ('良い', 9751),
 ('ポケモン', 7850),
 ('言う', 7581),
 ('行く', 6897),
 ('来る', 6598),
 ('無い', 6223),
 ('そう', 5866),
 ('遣る', 5856),
 ('居る', 5802),
 ('成る', 5640),
 ('凄い', 5622),
 ('事', 5461),
 ('皆', 4971),
 ('見る', 4854),
 ('有る', 4792),
 ('どう', 4636),
 ('外', 4571),
 ('人', 3761),
 ('一寸', 3655),
 ('もう', 3494),
 ('ピカチュウ', 2667),
 ('暴れる', 2551),
 ('バトル', 2540),
 ('呉れる', 2492),
 ('分かる', 2383),
 ('今', 2356),
 ('出来る', 2321),
 ('マイク', 2317),
 ('サトシ', 2290),
 ('本当', 2087),
 ('待つ', 1954),
 ('出る', 1927),
 ('思う', 1913),
 ('今日', 1885),
 ('可愛い', 1804),
 ('下さる', 1720),
 ('技', 1704),
 ('持つ', 1513),
 ('時', 1490),
 ('頑張る', 1473),
 ('入る', 1428),
 ('強い', 1426),
 ('御座る', 1409),
 ('後', 1400),
 ('ぴか', 1388),
 ('こう', 1318),
 ('ロト', 1284),
 ('早い', 1270),
 ('タイプ', 1249),
 ('一', 1223),
 ('所', 1196),
 ('物', 1181),
 ('ゲット', 1170),
 ('格好', 1150),
 ('貰う', 1139),
 ('使う', 1136),
 ('矢張り', 1131),
 ('一緒', 1116),
 ('やばい', 1042),
 ('願う', 1021),
 ('交換', 985),
 ('未だ', 960),
 ('怖い', 951),
 ('方', 929),
 ('気', 917),
 ('イケザキ

In [30]:
df_voca = pd.DataFrame(voca_list)
df_voca.columns = ["japanese", "frequency"]
df_voca

,japanese,frequency
0,為る,19535
1,良い,9751
2,ポケモン,7850
3,言う,7581
4,行く,6897
...,...,...
995,レース,85
996,コース,85
997,野生,85
998,吹く,85


In [31]:
word_list = []
df_voca.apply(save_word, axis=1)
dbcol_ani_voca.update_one({"series_id": pocket_series}, {"$set": {"ani_id": pocket_id, "words": word_list}}, upsert = True)

In [32]:
pocket_voca = dbcol_ani_voca.find_one({"series_id": pocket_series})["words"]
pocket_voca

[{'japanese': '為る', 'frequency': 19535},
 {'japanese': '良い', 'frequency': 9751},
 {'japanese': 'ポケモン', 'frequency': 7850},
 {'japanese': '言う', 'frequency': 7581},
 {'japanese': '行く', 'frequency': 6897},
 {'japanese': '来る', 'frequency': 6598},
 {'japanese': '無い', 'frequency': 6223},
 {'japanese': 'そう', 'frequency': 5866},
 {'japanese': '遣る', 'frequency': 5856},
 {'japanese': '居る', 'frequency': 5802},
 {'japanese': '成る', 'frequency': 5640},
 {'japanese': '凄い', 'frequency': 5622},
 {'japanese': '事', 'frequency': 5461},
 {'japanese': '皆', 'frequency': 4971},
 {'japanese': '見る', 'frequency': 4854},
 {'japanese': '有る', 'frequency': 4792},
 {'japanese': 'どう', 'frequency': 4636},
 {'japanese': '外', 'frequency': 4571},
 {'japanese': '人', 'frequency': 3761},
 {'japanese': '一寸', 'frequency': 3655},
 {'japanese': 'もう', 'frequency': 3494},
 {'japanese': 'ピカチュウ', 'frequency': 2667},
 {'japanese': '暴れる', 'frequency': 2551},
 {'japanese': 'バトル', 'frequency': 2540},
 {'japanese': '呉れる', 'frequency': 24

In [33]:
result = []
def make_result(data):
    for w in data:
        j = w["japanese"]
        f = w["frequency"]
        word = dbcol_words.find_one({"japanese": j})
        if word["korean"] != word["papago"]:
            word["korean"] = word["korean"] + ", " + word["papago"]
        word = {
            "japanese": word["japanese"],
            "pronunciation": word["pronunciation"],
            "korean": word["korean"],
            "frequency": f
        }
        result.append(word)
    return result

result = make_result(pocket_voca)
result

[{'japanese': '為る',
  'pronunciation': 'Naru',
  'korean': '하다',
  'frequency': 19535},
 {'japanese': '良い',
  'pronunciation': 'Yoi',
  'korean': '좋은, 좋다',
  'frequency': 9751},
 {'japanese': 'ポケモン',
  'pronunciation': 'Pokemon',
  'korean': '포켓몬',
  'frequency': 7850},
 {'japanese': '言う', 'pronunciation': 'Iu', 'korean': '말하다', 'frequency': 7581},
 {'japanese': '行く', 'pronunciation': 'Iku', 'korean': '가다', 'frequency': 6897},
 {'japanese': '来る',
  'pronunciation': 'Kuru',
  'korean': '오다',
  'frequency': 6598},
 {'japanese': '無い',
  'pronunciation': 'Nai',
  'korean': '아니, 없다',
  'frequency': 6223},
 {'japanese': 'そう',
  'pronunciation': 'Sō',
  'korean': '예, 그래',
  'frequency': 5866},
 {'japanese': '遣る',
  'pronunciation': 'Yaru',
  'korean': '보내다, 주다',
  'frequency': 5856},
 {'japanese': '居る',
  'pronunciation': 'Iru',
  'korean': '~이다, 있다',
  'frequency': 5802},
 {'japanese': '成る',
  'pronunciation': 'Naru',
  'korean': '이 되다, 이루어지다',
  'frequency': 5640},
 {'japanese': '凄い',
  'pr

In [34]:
dbcol_ani_voca.update_one({"series_id": pocket_series}, {"$set": {"words": result}})

## ✨ 세일러문

In [36]:
query = {
        "filename": {"$regex": "Sailor"}
}
df_words = pd.DataFrame(dbcol_subtitle.find(query))
df_words = df_words[["frequency"]]

# 세일러문
series_id = 3478
df_ani = pd.DataFrame(dbcol_info.find({"series_id": series_id}))

ani_id = df_ani.id.values.tolist()

In [80]:
voca = dict()
def init_words(data):
    frequency = data["frequency"]
    for word, cnt in frequency.items():
        voca[word] = 0
    return voca

def count_words(data, voca=voca):
    frequency = data["frequency"]
    for word, cnt in frequency.items():
        voca[word] += cnt
    return voca

df_words.apply(init_words, axis=1)
df_words.apply(count_words, axis=1)

voca_list = sorted(voca.items(), key=lambda x: x[1], reverse=True)[:1000]

df_voca = pd.DataFrame(voca_list)
df_voca.columns = ["japanese", "frequency"]
df_voca

,japanese,frequency
0,為る,6487
1,無い,3200
2,事,2372
3,居る,2280
4,良い,2237
...,...,...
995,パーティー,24
996,我が儘,24
997,ファイター,24
998,ガラス,23


In [87]:
word_list = []
df_voca.apply(save_word, axis=1)
dbcol_ani_voca.update_one({"series_id": series_id}, {"$set": {"ani_id": ani_id, "words": word_list}}, upsert = True)
ani_voca = dbcol_ani_voca.find_one({"series_id": series_id})["words"]
ani_voca

[{'japanese': '為る', 'frequency': 6487},
 {'japanese': '無い', 'frequency': 3200},
 {'japanese': '事', 'frequency': 2372},
 {'japanese': '居る', 'frequency': 2280},
 {'japanese': '良い', 'frequency': 2237},
 {'japanese': '言う', 'frequency': 2115},
 {'japanese': 'セーラー', 'frequency': 1951},
 {'japanese': '兎', 'frequency': 1723},
 {'japanese': '成る', 'frequency': 1639},
 {'japanese': 'そう', 'frequency': 1606},
 {'japanese': 'どう', 'frequency': 1560},
 {'japanese': 'ムーン', 'frequency': 1542},
 {'japanese': '人', 'frequency': 1381},
 {'japanese': '有る', 'frequency': 1374},
 {'japanese': '行く', 'frequency': 1359},
 {'japanese': '来る', 'frequency': 1310},
 {'japanese': 'もう', 'frequency': 1258},
 {'japanese': 'ちび', 'frequency': 1193},
 {'japanese': '皆', 'frequency': 1128},
 {'japanese': '遣る', 'frequency': 986},
 {'japanese': '憂さ', 'frequency': 979},
 {'japanese': '見る', 'frequency': 889},
 {'japanese': '夢', 'frequency': 806},
 {'japanese': '分かる', 'frequency': 792},
 {'japanese': '物', 'frequency': 772},
 {'japan

In [90]:
result = []
def make_result(data):
    for w in data:
        j = w["japanese"]
        f = w["frequency"]
        word = dbcol_words.find_one({"japanese": j})
        print(j)
        if word["korean"] != word["papago"]:
            word["korean"] = word["korean"] + ", " + word["papago"]
        word = {
            "japanese": word["japanese"],
            "pronunciation": word["pronunciation"],
            "korean": word["korean"],
            "frequency": f
        }
        result.append(word)
    return result

result = make_result(ani_voca)
result

為る
無い
事
居る
良い
言う
セーラー
兎
成る
そう
どう
ムーン
人
有る
行く
来る
もう
ちび
皆
遣る
憂さ
見る
夢
分かる
物
待つ
呉れる
為さる
今
思う
戦士
本当
月
持つ
一寸
駄目
子
レイ
出来る
時
アミ
心
早い
矢張り
方
所
愛
間
奴
水晶
止める
パワー
御免
変わる
力
上げる
ルナ
仕舞う
下さる
知る
ふっ
為
エネルギー
仮面
気
悪い
マモル
貰う
タキシード
今日
世界
蛍
ミナコ
許す
手
一
訳
スター
ママ
仕置き
未だ
又
会う
守る
返る
ペガサス
星
プリンセス
一緒
美しい
聞く
自分
中
マコ
出る
馬鹿
者
助ける
敵
銀
凄い
願う
少女
一体
良く
違う
前
見せる
こう
勉強
クリスタル
姉
死ぬ
急度
後
うう
友達
ピュア
頑張る
邪魔
目
信ずる
正義
食べる
確り
可愛い
侭
もっと
子供
出す
未来
女の子
光
妖魔
美
今度
まさか
一人
人間
頂く
地球
変
服
全て
内
使う
度
入る
筈
作る
絶対
先生
探す
まあ
男
ホシノ
戦う
若し
逃げる
少し
恋
取る
直ぐ
変身
嘘
気持ち
怖い
愛する
入れる
話
日
置く
団子
真
番
童
考える
付ける
任せる
心配
マーキュリー
ずっと
現われる
年
全く
何故
知れる
ウンノ
相手
時間
痛い
兄
ハルカ
済む
乙女
体
呼ぶ
パパ
忘れる
積り
ちびちび
頭
女
奪う
明日
シード
マーズ
格好
ちゃんと
見付ける
アルテミス
強い
通り
元気
御座る
感ずる
先
危ない
レディー
猫
欲しい
美味しい
ああ
必ず
命
戻る
仲間
見える
ジュピター
教える
過ぎる
姿
はあはあ
付く
消える
次
ライト
花
どうぞ
さっき
別
酷い
他
掛ける
必要
闇
学校
ゴールデン
爺
大気
虹
銀河
一杯
返す
ダーク
音
煩い
無理
兎に角
失礼
折角
迚も
幸せ
娘
恋人
楽しい
遊ぶ
ミチル
話す
負ける
終わる
家
倒す
新しい
十
泣く
仕事
宜しく
急ぐ
関係
モトキ
出でる
決まる
生きる
唯
顔
失敗
声
ビーナス
ツキノ
以上
暗黒
うわ
嬉しい
大人
怒る
渡す
宇宙
狙う
サトル
雄
スモール
正体
所為
諦める
アイドル
鏡
買う
遅い
甘い
ミズノ
黙る
母
勝つ
幻
全然
乗る
最後
優し

[{'japanese': '為る',
  'pronunciation': 'Naru',
  'korean': '하다',
  'frequency': 6487},
 {'japanese': '無い',
  'pronunciation': 'Nai',
  'korean': '아니, 없다',
  'frequency': 3200},
 {'japanese': '事',
  'pronunciation': 'Koto',
  'korean': '사례, 일',
  'frequency': 2372},
 {'japanese': '居る',
  'pronunciation': 'Iru',
  'korean': '~이다, 있다',
  'frequency': 2280},
 {'japanese': '良い',
  'pronunciation': 'Yoi',
  'korean': '좋은, 좋다',
  'frequency': 2237},
 {'japanese': '言う', 'pronunciation': 'Iu', 'korean': '말하다', 'frequency': 2115},
 {'japanese': 'セーラー',
  'pronunciation': 'Sērā',
  'korean': '선원, 세일러',
  'frequency': 1951},
 {'japanese': '兎',
  'pronunciation': 'Usagi',
  'korean': '토끼',
  'frequency': 1723},
 {'japanese': '成る',
  'pronunciation': 'Naru',
  'korean': '이 되다, 이루어지다',
  'frequency': 1639},
 {'japanese': 'そう',
  'pronunciation': 'Sō',
  'korean': '예, 그래',
  'frequency': 1606},
 {'japanese': 'どう',
  'pronunciation': 'Dō',
  'korean': '구리, 어떻게',
  'frequency': 1560},
 {'japanese': 'ムーン

In [91]:
dbcol_ani_voca.update_one({"series_id": series_id}, {"$set": {"words": result}})